<a href="https://colab.research.google.com/github/DanishGada/GANS/blob/main/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import imageio
import numpy as np
import matplotlib
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook

In [ ]:
batch_size = 1024
epochs = 200
sample_size = 64 # fixed sample size
nz = 128 # latent vector size
k = 1 # number of steps to apply to the discriminator

In [ ]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])
to_pil_image = transforms.ToPILImage()

In [ ]:
train_data = datasets.MNIST(
    root='../input/data',
    train=True,
    download=True,
    transform=transform
)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.nz = nz
        self.main = nn.Sequential(
            nn.Linear(self.nz, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_input = 784
        self.main = nn.Sequential(
            nn.Linear(self.n_input, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        x = x.view(-1, 784)
        return self.main(x)

In [ ]:
generator = Generator(nz)
discriminator = Discriminator()
print('##### GENERATOR #####')
print(generator)
print('######################')
print('\n##### DISCRIMINATOR #####')
print(discriminator)
print('######################')

In [ ]:
optim_g = optim.Adam(generator.parameters(), lr=0.0002)
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
losses_g = [] 
losses_d = [] 
images = [] 

In [ ]:
def label_real(size):
    data = torch.ones(size, 1)
    return data
    
def label_fake(size):
    data = torch.zeros(size, 1)
    return data

In [ ]:
def create_noise(sample_size, nz):
    return torch.randn(sample_size, nz)

In [ ]:
def save_generator_image(image, path):
    save_image(image, path)

In [ ]:
def train_discriminator(optimizer, data_real, data_fake):
    b_size = data_real.size(0)
    real_label = label_real(b_size)
    fake_label = label_fake(b_size)
    optimizer.zero_grad()
    # sEND REAL data
    output_real = discriminator(data_realx`)
    loss_real = criterion(output_real, real_label)
    #Send fake data
    output_fake = discriminator(data_fake)
    loss_fake = criterion(output_fake, fake_label)
    #backprop
    loss_real.backward()
    loss_fake.backward()
    optimizer.step()
    return loss_real + loss_fake

In [ ]:
def train_generator(optimizer, data_fake):
    b_size = data_fake.size(0)
    real_label = label_real(b_size)
    optimizer.zero_grad()
    output = discriminator(data_fake)
    loss = criterion(output, real_label)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
noise = create_noise(sample_size, nz)

In [ ]:
generator.train()
discriminator.train()

In [ ]:
epoch_loss_g=0
epoch_loss_d=0

'''
        In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch 
        accumulates the gradients on subsequent backward passes. This is convenient while training RNNs. So, 
        the default action is to accumulate (i.e. sum) the gradients on every loss.backward() call.
        
        Because of this, when you start your training loop, ideally you should zero out the gradients so that 
        you do the parameter update correctly. Else the gradient would point in some other direction than the 
        intended direction towards the minimum (or maximum, in case of maximization objectives).
        
        '''
for epoch in range(epochs):
    loss_g = 0.0
    loss_d = 0.0
    for bi, data in enumerate(train_loader):
        image, _ = data
        b_size = len(image)
        if epoch_loss_g > 3 and abs(epoch_loss_d - 0.5) < 0.05:
          k=1
        else:
          k=2
        for step in range(k):
            data_fake = generator(create_noise(b_size, nz)).detach()
            data_real = image
        
        loss_d += train_discriminator(optim_d, data_real, data_fake)
        data_fake = generator(create_noise(b_size, nz))
        loss_g += train_generator(optim_g, data_fake)
        print('.',end="")
    generated_img = generator(noise)
    generated_img = make_grid(generated_img)
    save_generator_image(generated_img, f"/content/drive/MyDrive/Colab Notebooks/outputs2/gen_img{epoch}.png")
    images.append(generated_img)
    epoch_loss_g = loss_g / bi 
    epoch_loss_d = loss_d / bi
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)
    
    print(f"Epoch {epoch + 1} of {epochs}")
    print(f"Generator loss: {epoch_loss_g:}, Discriminator loss: {epoch_loss_d:}")